Берём английское слово - все связанные русские смыслослова - переводим их и храним как перечень  вариантов. 
Далее переводим анг предложение на локальный язык и смотрим с какое местное слово превалирует в переведённых предложении. Его и ставим смыслослово рассматриваемого примера 

In [ ]:
Осложнения этого метода
1) нужно будет искать предобученные w2v для каждого языка -- https://fasttext.cc/docs/en/crawl-vectors.html
2)нужно будет искать лемматизаторы для каждого языка
.....
если чего-либо не будет то на таких примерах откатываемся к наивному методу 

In [1]:
import json
import requests
import copy
from tqdm import tqdm

In [20]:
from string import punctuation
full_punctuation = punctuation + "–" + "," + "»" + "«" + "…" +'’' + '—'

In [11]:
def yandex_translate(text, from_lang, to_lang):
    url_y = "http://192.168.122.13:31436/GetTranslate"
    h_y = {"apiVersion": "1.0.0", "text": text, "langPair": {"source": from_lang, "target": to_lang}}
    r = requests.post(url = url_y, json=h_y)
    data = r.json()
    if 'translate' in data:
        return data['translate']
    else:
        return None

In [3]:
with open("big_dict_with_almost_all_features_v2.json", "r") as f:
    big_dict_with_almost_all_features = json.load(f)

In [26]:
def get_local_word_from_context(local_sensewords_list, examples_list ):
    translated_examples = []
    for example in examples_list:
        translated_example = yandex_translate(example,"en", "fr")
        translated_examples.append(translated_example)
    print("translated_examples", translated_examples)
    print("local_sensewords_list", local_sensewords_list)

def translate_word_brnch(word_branch_sences_list):
    senseword_local_variants = []
    for lemma_sence_list_element in word_branch_sences_list:
        print(lemma_sence_list_element['local_word'])
        translate = yandex_translate(lemma_sence_list_element['local_word'],"ru", "fr")
        if translate:
            senseword_local_variants.append(translate)
    for lemma_sence_list_element in word_branch_sences_list:
        """
        if 'definition' in lemma_sence_list_element:
            lemma_sence_list_element['definition_local'] = y_translate(lemma_sence_list_element['definition'],"en", "fr")"""
        get_local_word_from_context(senseword_local_variants, lemma_sence_list_element['examples'])
        break
    
    """    
    for lemma_sence_list_element in word_branch_sences_list:
        if 'definition' in lemma_sence_list_element:
            lemma_sence_list_element['definition_local'] = y_translate(lemma_sence_list_element['definition'],"en", "fr")
        #print(lemma_sence_list_element['local_word'])
        lemma_sence_list_element['local_word'] = y_translate(lemma_sence_list_element['local_word'],"ru", "fr")
        translated_examples_list = []
        translated_count = 0 
        for example in lemma_sence_list_element['examples']:
            translated_example = y_translate(example,"en", "fr")
            translated_examples_list.append(translated_example)
            translated_count += 1
            if translated_count >= 3: break
        lemma_sence_list_element['examples_local'] = translated_examples_list """
        
def naive_translate(rus_dict):
    new_dict = copy.deepcopy(rus_dict)
    for eng_word in tqdm(new_dict.keys()):
        #print(eng_word)
        translate_word_brnch(new_dict[eng_word]['lemma_branch'])
        for wordform in new_dict[eng_word]['wordforms_branch'].keys(): 
            #print(wordform)
            translate_word_brnch(new_dict[eng_word]['wordforms_branch'][wordform]['wordsenses'])
            break
        break
        #print("======\n")  
    return new_dict
translated_dict = naive_translate(dct)




  0%|          | 0/1 [00:00<?, ?it/s]

тарелка
пластина, лист (металла, стекла)
позолота/серебрение
иллюстрация
translated_examples ['une assiette', 'une assiette de biscuits', "C'est la bouée de vie D'une énorme plaque de chips.", "Non, c'est juste une plaque de papier.", "Je veux dire, nous allons avoir besoin d'une autre plaque."]
local_sensewords_list ['assiette', 'plaque, feuille (métal, verre)', 'plaqué or / argent plaqué', 'illustration']
тарелки
пластин
номерами
номерные знаки
тарелок
табличек
номеров
пластинами
плит
плиты
translated_examples ['Luciole... les plaques enflammées du monde des insectes.', "Je vais devoir rentrer chez moi d'abord et sortir les assiettes de la salle de bain.", 'Vous pouvez faire des assiettes avec qui beaucoup.']
local_sensewords_list ['cymbale', 'plaques', 'chambres', "plaque d'immatriculation", 'assiettes', 'panonceaux', 'chambres', 'plaques', 'cuisinières', 'cuisinières']


In [18]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()

In [29]:
def local_lemmatizer(text):
    clean_text = ''
    for text_char in text:
        if text_char not in full_punctuation:
            clean_text += text_char.lower()
    lemm_list = [lemmatizer.lemmatize(word) for word in clean_text.split()]
    return lemm_list
local_lemmatizer("Luciole... les plaques enflammées du monde des insectes")

['luciole', 'les', 'plaque', 'enflammées', 'du', 'monde', 'des', 'insecte']

In [40]:
def get_highest_sence_intersection(one_senseword_list, lemmatized_local_ex_list):
    for senseword in one_senseword_list:
        for lemm_local_ex in lemmatized_local_ex_list:
            if senseword in lemm_local_ex:
                print (senseword, "in", lemm_local_ex)
                return 0
    print("No intersection")
        
        

In [41]:
potential_sencewords = ['cymbale', 'plaque', 'chambres', "la plaque d'immatriculation", 'assiettes', 'panonceaux', 'chambres', 'plaque', 'cuisinières', 'cuisinières']
translated_examples = ['Luciole... les plaques enflammées du monde des insectes.', "Je vais devoir rentrer chez moi d'abord et sortir les assiettes de la salle de bain.", 'Vous pouvez faire des assiettes avec qui beaucoup.']
def get_local_word_from_context(local_sensewords_list, examples_list):
    
    translated_examples = examples_list
    """translated_examples = []
    for example in examples_list:
        translated_example = yandex_translate(example,"en", "fr")
        translated_examples.append(translated_example)
    print(translated_examples)
    print(local_sensewords_list)"""
    lemm_ex_list = []
    for tr_ex in translated_examples:
        lemm_example = local_lemmatizer(tr_ex)
        lemm_ex_list.append(lemm_example)
    print(lemm_ex_list)
    for local_senseword in local_sensewords_list:
        lemm_senseword = local_lemmatizer(local_senseword)
        print(lemm_senseword)
        get_highest_sence_intersection(lemm_senseword, lemm_ex_list)
        
        
    
get_local_word_from_context(potential_sencewords,translated_examples ) 
    

[['luciole', 'les', 'plaque', 'enflammées', 'du', 'monde', 'des', 'insecte'], ['je', 'vais', 'devoir', 'rentrer', 'chez', 'moi', 'dabord', 'et', 'sortir', 'les', 'assiette', 'de', 'la', 'salle', 'de', 'bain'], ['vous', 'pouvez', 'faire', 'des', 'assiette', 'avec', 'qui', 'beaucoup']]
['cymbale']
No intersection
['plaque']
plaque in ['luciole', 'les', 'plaque', 'enflammées', 'du', 'monde', 'des', 'insecte']
['chambre']
No intersection
['la', 'plaque', 'dimmatriculation']
la in ['je', 'vais', 'devoir', 'rentrer', 'chez', 'moi', 'dabord', 'et', 'sortir', 'les', 'assiette', 'de', 'la', 'salle', 'de', 'bain']
['assiette']
assiette in ['je', 'vais', 'devoir', 'rentrer', 'chez', 'moi', 'dabord', 'et', 'sortir', 'les', 'assiette', 'de', 'la', 'salle', 'de', 'bain']
['panonceau']
No intersection
['chambre']
No intersection
['plaque']
plaque in ['luciole', 'les', 'plaque', 'enflammées', 'du', 'monde', 'des', 'insecte']
['cuisinière']
No intersection
['cuisinière']
No intersection


In [9]:
dct = {"plate": {
        "word_id": 30934,
        "lemma_branch": [
            {
                "definition": "a flat, round object that is used for putting food on",
                "local_word": "тарелка",
                "examples": [
                    "a dinner plate",
                    "a plate of biscuits",
                    "This is the life - bit of Shaky, enormous plate of chips.",
                    "No, it's just a paper plate.",
                    "I mean, we'll need another plate."
                ],
                "definition_local": "плоский, круглый объект, который используется для размещения пищи на",
                "examples_local": [
                    "обеденная тарелка",
                    "тарелка печенья",
                    "Это жизненный бит шаткой, огромной тарелки чипсов.",
                    "Нет, это просто бумажная тарелка.",
                    "В смысле, нам нужна еще одна тарелка."
                ],
                "word_rating": 19295
            },
            {
                "definition": "a flat piece of metal or glass",
                "local_word": "пластина, лист (металла, стекла)",
                "examples": [
                    "I had a metal plate put in my knee after the accident."
                ],
                "definition_local": "плоский кусок металла или стекла",
                "examples_local": [
                    "После аварии мне в колено вставили металлическую пластину."
                ],
                "word_rating": 4384
            },
            {
                "definition": "metal with a thin layer of gold or silver on top",
                "local_word": "позолота/серебрение",
                "examples": [],
                "definition_local": "металл с тонким слоем золота или серебра поверх",
                "examples_local": [],
                "word_rating": 0
            },
            {
                "definition": "a picture in a book",
                "local_word": "иллюстрация",
                "examples": [],
                "definition_local": "картинка в книге",
                "examples_local": [],
                "word_rating": 0
            }
        ],
        "wordforms_branch": {
            "plates": {
                "word_id": 50715,
                "wordsenses": [
                    {
                        "local_word": "тарелки",
                        "examples": [
                            "Fireflies... the flaming plates of the insect world.",
                            "I'll just have to run home first and take some plates out of the bathroom.",
                            "You can make dinner plates with that lot."
                        ],
                        "examples_local": [
                            "Светляки... пылающие пластины мира насекомых.",
                            "Мне просто нужно сначала сбегать домой и взять несколько тарелок из ванной.",
                            "Вы можете сделать обеденные тарелки с этой партией."
                        ],
                        "word_rating": 10411
                    },
                    {
                        "local_word": "пластин",
                        "examples": [
                            "I accidentally scratched one of the glass plates with one of my rings.",
                            "It s used for the quick simultaneous erasure of the plates throughout of their lengths.",
                            "It is also used for the storage of the plates at the intervals between exposures."
                        ],
                        "examples_local": [
                            "Я случайно поцарапал одну из стеклянных пластин одним из моих колец.",
                            "Используется для быстрого одновременного стирания пластин по всей их длине.",
                            "Он также используется для хранения пластин в интервалах между экспозициями."
                        ],
                        "word_rating": 656
                    },
                    {
                        "local_word": "номерами",
                        "examples": [
                            "Suspects are traveling in a gold minivan with Texas license plates.",
                            "He's looking for a white convertible with Madrid plates.",
                            "I saw your car with the Illinois plates."
                        ],
                        "examples_local": [
                            "Подозреваемые путешествуют в золотом микроавтобусе с техасскими номерами.",
                            "Он ищет белый кабриолет с мадридскими номерами.",
                            "Я видел вашу машину с иллинойскими номерами."
                        ],
                        "word_rating": 545
                    },
                    {
                        "local_word": "номерные знаки",
                        "examples": [
                            "A concerned citizen tried to catch some of their license plates on camera.",
                            "Kosovo license plates are still not recognized by the Serbian Government, which exacerbates limitations on freedom of movement for the Kosovo Serb population.",
                            "UNMIK-issued Kosovo licence plates and registration are now also recognized by the former Yugoslav Republic of Macedonia."
                        ],
                        "examples_local": [
                            "Обеспокоенный гражданин попытался поймать некоторые из их номерных знаков на камеру.",
                            "Косовские номерные знаки по-прежнему не признаются сербским правительством, что усугубляет ограничения на свободу передвижения для населения косовских сербов.",
                            "Выданные МООНК косовские номерные знаки и регистрация теперь также признаются бывшей югославской республикой Македонией."
                        ],
                        "word_rating": 0
                    },
                    {
                        "local_word": "тарелок",
                        "examples": [
                            "And we want to sell thousands of those plates.",
                            "I carry too many plates and drinks at the restaurant.",
                            "A few years and whole lot of enchilada plates, a whole lot."
                        ],
                        "examples_local": [
                            "И мы хотим продать тысячи таких пластинок.",
                            "Я ношу слишком много тарелок и напитков в ресторане.",
                            "Несколько лет и целая куча тарелок энчилады, целая куча."
                        ],
                        "word_rating": 2110
                    },
                    {
                        "local_word": "табличек",
                        "examples": [
                            "Alternating such loads and deliveries, coupled with the requirement to change plates, is not easy to manage.",
                            "On the monument there are many plates with words of gratitude to this great Bolivian.",
                            "This issue has been addressed in ATP by the introduction of ATP plates which are allowed to replace the certificate and markings on small containers."
                        ],
                        "examples_local": [
                            "Чередовать такие нагрузки и поставки, соединенные с требованием изменить плиты, не легок для того чтобы управлять.",
                            "На памятнике много табличек со словами благодарности этому великому боливийцу.",
                            "Этот вопрос был урегулирован в СПС путем введения табличек СПС, которые могут заменять свидетельство и маркировку на малых контейнерах."
                        ],
                        "word_rating": 0
                    },
                    {
                        "local_word": "номеров",
                        "examples": [
                            "On 13 January, the Kosovo police began confiscating Serbian plates issued for Kosovo."
                        ],
                        "examples_local": [
                            "13 января косовская полиция начала конфискацию сербских номерных знаков, выданных для Косово."
                        ],
                        "word_rating": 0
                    },
                    {
                        "local_word": "пластинами",
                        "examples": [
                            "The main control unit shall be positioned within the central one third of the longitudinal, vertical and transverse dimensions of the space between the parallel plates.",
                            "Looks like they welded metal plates over the windows.",
                            "The inventive distance lattice comprises cells which are used for mounting the fuel elements or guide channels and are formed by perpendicular crossing plates."
                        ],
                        "examples_local": [
                            "Главный блок управления должен располагаться в пределах центральной трети продольного, вертикального и поперечного размеров пространства между параллельными пластинами.",
                            "Похоже, они сварили металлические пластины на окнах.",
                            "Сущность изобретения: дистанционная решетка содержит ячейки, которые используются для монтажа ТВЭЛов или направляющих каналов и образованы перпендикулярными пересекающимися пластинами."
                        ],
                        "word_rating": 0
                    },
                    {
                        "local_word": "плит",
                        "examples": [
                            "The geology and landscape of Indonesia were formed millions of years ago by the collision of three geodesic plates.",
                            "This is nature, it's tectonic plates shifting.",
                            "There are, as you know, 14 major tectonic plates."
                        ],
                        "examples_local": [
                            "Геология и ландшафт Индонезии сформировались миллионы лет назад в результате столкновения трех геодезических плит.",
                            "Это природа, это тектонические сдвиги плит.",
                            "Здесь, как известно, 14 крупных тектонических плит."
                        ],
                        "word_rating": 0
                    },
                    {
                        "local_word": "плиты",
                        "examples": [
                            "In the world economy, the tectonic plates are in motion and new actors are emerging.",
                            "The Earth's temperature is rising; tectonic plates are shifting; and lands are drying up.",
                            "It's like the tectonic plates have shifted."
                        ],
                        "examples_local": [
                            "В мировой экономике тектонические плиты находятся в движении и появляются новые субъекты.",
                            "Температура Земли повышается; тектонические плиты смещаются; и земли высыхают.",
                            "Как будто тектонические плиты сдвинулись."
                        ],
                        "word_rating": 0
                    }
                ]
            }
        }
    }
      }